In [14]:
from cellpose import models, io
from cellpose.io import *
from collections import defaultdict
import glob
import imagej
from jpype import JArray, JInt
import matplotlib.pyplot as plt
import os
import re
import pandas
from pandas import DataFrame
from pathlib import Path
import scyjava
import seaborn
import shutil
import tkinter as tk
from tkinter import filedialog
from PIL import Image
from zipfile import ZipFile
import csv
import numpy as np
from skimage.measure import find_contours, label
from jpype import JArray, JInt

In [2]:
scyjava.config.add_option('-Xmx30g') # this is to assign RAM to the java session
ij = imagej.init('/home/saka/sw/local/fiji2/2023', mode='interactive')
ij.ui().showUI()
ij.getVersion() #This is to make sure ImageJ/Fiji opened properly. In case of error, it could display '2.9.0/inactive' instead of the full version number

'2.14.0/1.54f'

In [3]:
# Dialog window to get the path of the working directory
root = tk.Tk()
root.withdraw()
# Set the path to the directory containing the images
directory_path = filedialog.askdirectory()

In [5]:
file_path = f"{directory_path}/C1-Experiment-1151.tif"
imgs = io.imread(file_path)
model = models.CellposeModel(gpu=True, model_type='hmdm_nuclei')
channels = [[0,0]]
#masks, flows, styles = model.eval(imgs, diameter=10, channels=channels)
masks, flows, styles = model.eval(imgs, diameter=91.206, channels=channels, do_3D=False, stitch_threshold=0.4)
io.save_masks(imgs, masks, flows, file_path, save_txt=True, png=False, tif=True)
io.masks_flows_to_seg(imgs, masks, flows, file_path, file_path)

z_axis not specified, assuming it is dim 0
if this is actually the channel_axis, use 'model.eval(channel_axis=0, ...)'
100%|█████████████████████████████████████████████| 8/8 [00:01<00:00,  7.46it/s]


In [8]:
seg_path = f"{directory_path}/C1-Experiment-1151_seg.npy"
data = np.load(seg_path, allow_pickle=True).item()  # Load as dictionary
mask = data["masks"]  

In [9]:
rm = ij.RoiManager.getRoiManager()

In [12]:
PolygonRoi = scyjava.jimport('ij.gui.PolygonRoi')
Roi = scyjava.jimport('ij.gui.Roi')
for z in range(mask.shape[0]):
    contours = find_contours(mask[z], 0.5)  # Extract object boundaries
    
    for contour in contours:
        x_points = contour[:, 1].astype(int)
        y_points = contour[:, 0].astype(int)
        xcoords_jint = JArray(JInt)(x_points)
        ycoords_jint = JArray(JInt)(y_points)
        # Create ImageJ ROI
        roi = PolygonRoi(xcoords_jint, ycoords_jint, len(x_points), Roi.FREEROI)
        roi.setPosition(z + 1)  # Assign Z slice position
        rm.addRoi(roi)



In [15]:
# Initialize a global object ID counter
object_id_map = {}  # Dictionary to track object IDs across slices
next_object_id = 1  # Start with ID 1

for z in range(mask.shape[0]):
    labeled_slice = label(mask[z])  # Assigns unique labels to objects in this slice
    unique_objects = np.unique(labeled_slice)
    unique_objects = unique_objects[unique_objects > 0]  # Ignore background (0)

    for obj in unique_objects:
        # Find contours of this specific object
        object_mask = (labeled_slice == obj)
        contours = find_contours(object_mask, 0.5)

        for contour in contours:
            x_points = contour[:, 1].astype(int)
            y_points = contour[:, 0].astype(int)

            # Convert NumPy arrays to Java arrays
            xcoords_jint = JArray(JInt)(x_points)
            ycoords_jint = JArray(JInt)(y_points)

            # Assign a persistent object ID across slices
            if obj not in object_id_map:
                object_id_map[obj] = next_object_id
                next_object_id += 1  # Increment for new objects

            object_id = object_id_map[obj]  # Get the assigned ID

            # Create ImageJ ROI
            roi = PolygonRoi(xcoords_jint, ycoords_jint, len(x_points), Roi.FREEROI)
            roi.setPosition(z + 1)  # Assign Z slice position

            # Rename ROI to "objX_sliceY"
            roi.setName(f"obj{object_id}_slice{z+1}")

            # Add ROI to the ROI Manager
            rm.addRoi(roi)